In [1]:
#!pip install tf-nightly-gpu
#!pip install "tensorflow_hub==0.4.0"
#!pip install tensorflow

In [2]:
# for colab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
# standart ds libs
import random
import numpy as np 
import pandas as pd
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'
import sys
import PIL
import re
import json
from pandas.io.json import json_normalize
import pickle

# plt
import matplotlib.pyplot as plt
import scipy.stats as stats
import pylab 
#change deff size
from pylab import rcParams
rcParams['figure.figsize'] = 25, 15
plt.rcParams["figure.figsize"] = (25,25)
#graps in svg
%config InlineBackend.figure_format = 'svg' 
%matplotlib inline
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)

import seaborn as sns
sns.set_theme(style="whitegrid")

In [4]:
image = PIL.Image

In [5]:
# ml libs
from __future__ import absolute_import, division, print_function, unicode_literals

import matplotlib.pylab as plt

import tensorflow as tf

import tensorflow_hub as hub

from tensorflow.keras import layers

import tensorflow.compat.v1.keras.backend as K
import tensorflow as tf
tf.compat.v1.disable_eager_execution()

In [6]:
tf.executing_eagerly()

False

In [7]:
import logging
logger = tf.get_logger()
logger.setLevel(logging.ERROR)

In [8]:
CLASSIFIER_URL = "https://tfhub.dev/google/tf2-preview/mobilenet_v2/classification/4"
IMAGE_RES = 224
'''
model = tf.keras.Sequential([
                             hub.KerasLayer(CLASSIFIER_URL, trainable=False, input_shape=(IMAGE_RES, IMAGE_RES, 3))
])'''

'\nmodel = tf.keras.Sequential([\n                             hub.KerasLayer(CLASSIFIER_URL, trainable=False, input_shape=(IMAGE_RES, IMAGE_RES, 3))\n])'

In [9]:
'''model'''

'model'

Sample

In [10]:
# use local
#DATA_DIR = 'Dataset/'
#target.head(2)

In [11]:
# use on colab
DATA_DIR = '/content/drive/MyDrive/COLAB_DATA/Dataset/' 
colab_path = '/content/drive/MyDrive/COLAB_DATA/target_table.pickle'
!pip3 install pickle5
import pickle5 as pickle
with open(colab_path, "rb") as fh:
  target = pickle.load(fh)

In [12]:
target.head(2)

,file,target
0,data/bf0d8c12-950b-4760-8deb-3a562cd7fe5b.jpg,0
1,data/021d56b2-305b-443a-b45b-17ca7eafc3e9.jpg,0


In [13]:
#loss_weiths = {}
loss_weiths = []
for i in list(target.target.unique()):
    #loss_weiths[i] = 1- (len(target[target.target == i]) / len(target))
    loss_weiths.append(1- (len(target[target.target == i]) / len(target)))
loss_weiths  = np.array(loss_weiths)  
loss_weiths.shape

(3,)

In [14]:
'''random_image = target.sample(n = 1)
random_image_path = random_image['file'].values[0]
random_image_type = random_image['target'].values[0]
sample_pic = image.open(DATA_DIR+random_image_path).convert('RGB').resize((IMAGE_RES, IMAGE_RES))
print(random_image_type, random_image_path)
sample_pic'''

"random_image = target.sample(n = 1)\nrandom_image_path = random_image['file'].values[0]\nrandom_image_type = random_image['target'].values[0]\nsample_pic = image.open(DATA_DIR+random_image_path).convert('RGB').resize((IMAGE_RES, IMAGE_RES))\nprint(random_image_type, random_image_path)\nsample_pic"

In [15]:
'''random_image_path'''

'random_image_path'

In [16]:
'''sample_ar = np.array(sample_pic)/255.0
sample_ar.shape'''

'sample_ar = np.array(sample_pic)/255.0\nsample_ar.shape'

In [17]:
'''result = model.predict(sample_ar[np.newaxis, ...])
result.shape'''

'result = model.predict(sample_ar[np.newaxis, ...])\nresult.shape'

In [18]:
'''predicted_class = np.argmax(result[0], axis=-1)
predicted_class'''

'predicted_class = np.argmax(result[0], axis=-1)\npredicted_class'

In [19]:
'''labels_path = tf.keras.utils.get_file('ImageNetLabels.txt','https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt')
imagenet_labels = np.array(open(labels_path).read().splitlines())

plt.imshow(sample_pic)
plt.axis('off')
predicted_class_name = imagenet_labels[predicted_class]
_ = plt.title("Prediction: " + predicted_class_name.title())'''

'labels_path = tf.keras.utils.get_file(\'ImageNetLabels.txt\',\'https://storage.googleapis.com/download.tensorflow.org/data/ImageNetLabels.txt\')\nimagenet_labels = np.array(open(labels_path).read().splitlines())\n\nplt.imshow(sample_pic)\nplt.axis(\'off\')\npredicted_class_name = imagenet_labels[predicted_class]\n_ = plt.title("Prediction: " + predicted_class_name.title())'

сделаем датафрейм фото в RGB + class

In [20]:
target_class ={0:'other'
              ,1:'vehicles'
              ,2:'plants'}  
 
# ресайз и приведеине к ргб
def getPic(row):
    return np.array(image.open(DATA_DIR + str(row['file'])).convert('RGB').resize((IMAGE_RES, IMAGE_RES)))

                    
# таргет класс
def get_label(row):
    return row['target']

# собираем Pics n Classes в np.array 
def get_ds(df):
    img_paths = list()
    # пустой дф
    for index, row in df.iterrows():
        img_paths.append(row)
    images = np.zeros((len(img_paths),IMAGE_RES,IMAGE_RES,3))
    labels = np.zeros(len(img_paths))
      
    # заполняем
    for i, row in enumerate(img_paths):
        images[i] = getPic(row)
        labels[i] = get_label(row)
        
    return images,labels
# Load the train and validation data
X, y = get_ds(target)

In [21]:
y[122]

2.0

Y to dummy

In [22]:
Y = pd.DataFrame({'y':y})
Y['0']=Y['y'].apply(lambda x: 1 if x== 0 else 0)
Y['1']=Y['y'].apply(lambda x: 1 if x== 1 else 0)
Y['2']=Y['y'].apply(lambda x: 1 if x== 2 else 0)
Y = Y.drop('y', axis = 1)
#Y

Модифицируемодель для предсказания наших классов

In [23]:
f_extractor = tf.keras.Sequential([
                             hub.KerasLayer(CLASSIFIER_URL, trainable=False, input_shape=(IMAGE_RES, IMAGE_RES, 3))
])
f_extractor.trainable = False

In [24]:
model = tf.keras.Sequential([
  f_extractor,
  layers.Dense(3, activation='softmax')
])

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 sequential (Sequential)     (None, 1001)              3540265   
                                                                 
 dense (Dense)               (None, 3)                 3006      
                                                                 
Total params: 3,543,271
Trainable params: 3,006
Non-trainable params: 3,540,265
_________________________________________________________________


In [25]:
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy'],
)

In [26]:
#batches = tf.compat.v1.train.batch(X ,Y ,  num_threads=1,capacity = 64 )
# Падает на моем буке

In [27]:
print(int(len(y)/200))
print(len(y)/200)

7
7.79


In [28]:
s1= 0
s2 = 200
ran = s2
x_t = tf.convert_to_tensor(X[s1:s2])
y_t = tf.convert_to_tensor(y[s1:s2])
# Prepare the training dataset
train_dataset = tf.data.Dataset.from_tensor_slices((x_t, y_t))
train_dataset = train_dataset.shuffle(buffer_size=256).batch(16)
# Prepare the validation dataset
val_dataset = tf.data.Dataset.from_tensor_slices((x_t, y_t))
val_dataset = val_dataset.batch(16)
for i in range(3):
  del x_t
  del y_t
  del train_dataset
  del val_dataset
  x_t = tf.convert_to_tensor(X[s1:s2])
  y_t = tf.convert_to_tensor(y[s1:s2])
  # Prepare the training dataset
  train_dataset = tf.data.Dataset.from_tensor_slices((x_t, y_t))
  train_dataset = train_dataset.shuffle(buffer_size=256).batch(16)

  # Prepare the validation dataset
  val_dataset = tf.data.Dataset.from_tensor_slices((x_t, y_t))
  val_dataset = val_dataset.batch(16)

  model.fit(train_dataset,
            epochs=2,
            validation_data=val_dataset)

  s1 +=ran
  s2 +=ran

Train on 13 steps, validate on 13 steps
Epoch 1/2
13/13 [==============================] - ETA: 0s - batch: 6.0000 - size: 1.0000 - loss: 0.8262 - accuracy: 0.6900

/usr/local/lib/python3.7/dist-packages/keras/engine/training_v1.py:2057: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates = self.state_updates


13/13 [==============================] - 25s 2s/step - batch: 6.0000 - size: 1.0000 - loss: 0.8262 - accuracy: 0.6900 - val_loss: 0.3667 - val_accuracy: 0.8750
Epoch 2/2
13/13 [==============================] - 14s 1s/step - batch: 6.0000 - size: 1.0000 - loss: 0.3214 - accuracy: 0.8900 - val_loss: 0.2192 - val_accuracy: 0.9550
Train on 13 steps, validate on 13 steps
Epoch 1/2
13/13 [==============================] - 19s 1s/step - batch: 6.0000 - size: 1.0000 - loss: 0.4787 - accuracy: 0.8350 - val_loss: 0.3288 - val_accuracy: 0.8600
Epoch 2/2
13/13 [==============================] - 13s 1s/step - batch: 6.0000 - size: 1.0000 - loss: 0.3174 - accuracy: 0.8750 - val_loss: 0.2380 - val_accuracy: 0.9300
Train on 13 steps, validate on 13 steps
Epoch 1/2
13/13 [==============================] - 19s 1s/step - batch: 6.0000 - size: 1.0000 - loss: 0.3168 - accuracy: 0.8700 - val_loss: 0.2538 - val_accuracy: 0.8950
Epoch 2/2
13/13 [==============================] - 11s 868ms/step - batch: 6.000

In [32]:
model.save('traindel_model.h5')